In [ ]:
#Para versão recente do python
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 opencv-python mediapipe sklearn matplotlib

In [ ]:
!pip uninstall protobuf

In [ ]:
# Para versões antigas - https://www.python.org/ftp/python/3.7.4/
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python==4.5.2.54 mediapipe==0.8.5 sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
#MediaPipe solutions - reconhecimento e desenho dos pontos na mão
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    """ 
    Funcao com objetivo de aplicar a previsao de reconhecimento na imagem desejada 
    Recebe imagem (frame da webcam) e um modelo que sera responsavel pela previsão
    Retorna o frame utilizado e a previsao feita pelo modelo
    A conversao da imagem se faz necessaria para ser tratada pelo modelo (BGR -> RGB)
    A mudanca na propriedade de leitura tem como objetivo salvar memoria
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [5]:
def draw_landmarks(image, results):
    """
    Funcao que recebe uma imagem (frame) junto com as previsoes feitas pelo modelo e aplica
    sob a imagem o desenho dos pontos necessarios (nao ha motivo para devolver a imagem pois ela
    ja e alterada diretamente).
    """
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [7]:
def extract_keypoints(results, auto=False, axis=None, value=None):
    if auto:
        if axis == 'x':
            pose = np.array([[res.x+value, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
            face = np.array([[res.x+value, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
            lh = np.array([[res.x+value, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x+value, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
        elif axis == 'y':
            pose = np.array([[res.x, res.y+value, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
            face = np.array([[res.x, res.y+value, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
            lh = np.array([[res.x, res.y+value, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y+value, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
        #elif axis == 'z':
        #    pose = np.array([[res.x, res.y, res.z+value, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        #    face = np.array([[res.x, res.y, res.z+value] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
        #    lh = np.array([[res.x, res.y, res.z+value] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
        #    rh = np.array([[res.x, res.y, res.z+value] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
        else:
            auto = False
    else:
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [8]:
# Caminho que guarda o numpy array com os pontos extraidos
DATA_PATH = os.path.join('MP_Data')

# Sinais que serão detectados
actions = np.array(['ola', 'obrigado', 'teamo'])

# Representa a quantidade de sequências de frames que tem os dados
no_sequences = 30

# Representa a quantidade de frames que cada sequência possui
sequence_lenght = 30

In [9]:
# Cria pastas para cada sinal definido
# Cada pasta tem 30 pastas representando os videos modelo do sinal
# Cada vídeo contém 30 frames e cada frame 1662 pontos extraidos
for action in actions:
    for sequence in range(no_sequences):
        try: # Cria pastas e subpastas
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except: # Caso já exista passa para próxima pasta
            pass

In [ ]:
# TENTATIVA DE FAZER O PROGRAMA SIMULAR ENTRADAS    

#Acessa a webcam - valor (0) representa o hardware
cap = cv2.VideoCapture(0)

#Define o modelo utilizado
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Extrai os frames de cada video para cada ação
    for action in actions:
        for frame_num in range(sequence_lenght):
            #Seleciona o frame atual da webcam
            ret, frame = cap.read()
            
            #Processa previsoes
            image, results = mediapipe_detection(frame, holistic)
            #print(results)
            #Desenha os pontos
            draw_landmarks(image, results)
                
            if frame_num == 0:    
                #Textos para indicar inicio de gravação
                cv2.putText(image, "INICIANDO GRAVAÇÃO DE SINAL", 
                (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                
                cv2.putText(image, "Gravando para {} Frame: {}".format(action, frame_num), 
                (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                # Tempo de espera entre um vídeo e outro
                cv2.waitKey(2000)
            else:
                cv2.putText(image, "Gravando para {} Frame: {}".format(action, frame_num), 
                (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
            # Extrai os pontos de um frame e salva como um arquivo .npy (numpy array)
            for sequence in range(no_sequences):
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                if sequence < (no_sequences/2):
                    value_n = sequence*0.01
                    keypoints = extract_keypoints(results, auto=True, axis='x', value=value_n)
                elif sequence < (no_sequences):
                    value_n = (sequence-(no_sequences/2))*0.01
                    keypoints = extract_keypoints(results, auto=True, axis='y', value=value_n)
                np.save(npy_path, keypoints)
                    
                
            #Faz o display do frame
            if ret == True: 
                cv2.imshow('Webcam', image)
                #Encerra a webcam 
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            else:
                break
    cap.release()
    cv2.destroyAllWindows()

In [9]:
# Imports que serão importante para treinar o modelo e nomear os dados
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []

# Carrega e adiciona todos os pontos de uma sequência no array sequences
# Enquanto o array labels coloca em ordem o valor de determinada ação com certa sequencia
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_lenght):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence),'{}.npy'.format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    
    matrix_labels = to_categorical(labels).astype(int)
    X = np.array(sequences)
    
    # Com os dados já definidos é possível separar uma quantidade para teste e treinamento do modelo
    x_train, x_test, y_train, y_test = train_test_split(X, matrix_labels, test_size=0.05)

In [9]:
# Imports para construção da rede neural
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [10]:
# Definição do caminho para registro de logs da rede
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [11]:
# Setup da rede neural
model = Sequential()

# Cada função add adiciona uma layer para a rede
# O parâmetro return_sequences é importante para passar para próxima layer de forma ordenada
# O parâmetro activation define a função que será utilizada para ativar os neurônios
#  O parâmetro input_shape = 30 frames cada um com 1662 pontos reconhecidos
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) 
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Última camada possui a quantidades de pontos definido pela quantidade de ações
# A função softmax entrega um resultado com a probabilidade de cada ação (será selecionado a com maior probabilidade)
model.add(Dense(actions.shape[0], activation='softmax'))

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [27]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 6s 569ms/step - loss: 4.6893 - categorical_accuracy: 0.3639
Epoch 2/2000
3/3 [==============================] - 0s 92ms/step - loss: 6.7608 - categorical_accuracy: 0.2760
Epoch 3/2000
3/3 [==============================] - 0s 84ms/step - loss: 2.0025 - categorical_accuracy: 0.2897
Epoch 4/2000
3/3 [==============================] - 0s 89ms/step - loss: 1.1671 - categorical_accuracy: 0.3386
Epoch 5/2000
3/3 [==============================] - 0s 110ms/step - loss: 1.0937 - categorical_accuracy: 0.3659
Epoch 6/2000
3/3 [==============================] - 0s 96ms/step - loss: 1.1061 - categorical_accuracy: 0.3347
Epoch 7/2000
3/3 [==============================] - 0s 121ms/step - loss: 1.1002 - categorical_accuracy: 0.2642
Epoch 8/2000
3/3 [==============================] - 0s 92ms/step - loss: 1.0985 - categorical_accuracy: 0.3190
Epoch 9/2000
3/3 [==============================] - 0s 101ms/step - loss: 1.0980 - categorical_accuracy: 0.31

3/3 [==============================] - 0s 84ms/step - loss: 0.5427 - categorical_accuracy: 0.6908
Epoch 75/2000
3/3 [==============================] - 0s 93ms/step - loss: 0.5637 - categorical_accuracy: 0.6341
Epoch 76/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.5963 - categorical_accuracy: 0.6400
Epoch 77/2000
3/3 [==============================] - 0s 83ms/step - loss: 0.5336 - categorical_accuracy: 0.6986
Epoch 78/2000
3/3 [==============================] - 0s 80ms/step - loss: 0.5451 - categorical_accuracy: 0.7122
Epoch 79/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.5690 - categorical_accuracy: 0.6693
Epoch 80/2000
3/3 [==============================] - 0s 83ms/step - loss: 0.5952 - categorical_accuracy: 0.6204
Epoch 81/2000
3/3 [==============================] - 0s 81ms/step - loss: 0.7095 - categorical_accuracy: 0.6398
Epoch 82/2000
3/3 [==============================] - 0s 90ms/step - loss: 1.8262 - categorical_accuracy: 0.2271
Epoch 

3/3 [==============================] - 0s 81ms/step - loss: 0.2177 - categorical_accuracy: 0.8962
Epoch 219/2000
3/3 [==============================] - 0s 76ms/step - loss: 0.3700 - categorical_accuracy: 0.8982
Epoch 220/2000
3/3 [==============================] - 0s 79ms/step - loss: 1.4479 - categorical_accuracy: 0.5853
Epoch 221/2000
3/3 [==============================] - 0s 110ms/step - loss: 0.8426 - categorical_accuracy: 0.6184
Epoch 222/2000
3/3 [==============================] - 0s 109ms/step - loss: 0.7631 - categorical_accuracy: 0.6850
Epoch 223/2000
3/3 [==============================] - 0s 108ms/step - loss: 0.4891 - categorical_accuracy: 0.8355
Epoch 224/2000
3/3 [==============================] - 0s 125ms/step - loss: 0.5046 - categorical_accuracy: 0.7320
Epoch 225/2000
3/3 [==============================] - 0s 79ms/step - loss: 0.4115 - categorical_accuracy: 0.8336
Epoch 226/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.5337 - categorical_accuracy: 0

KeyboardInterrupt: 

In [28]:
# Mostra a performance do modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 64)            442112    
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
________________________________________________

In [18]:
# Retorna uma lista com valores de probabilidade de cada ação como definidos no modelo
res = model.predict(x_test)

In [30]:
# Salva os weights usados pelo modelo
model.save('action.h5')
#model.load_weights(action.h5)
#del model

In [13]:
model.load_weights('actionAC1.h5')

In [16]:
# Teste de métricas para aprovação do modelo
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [18]:
yhat = model.predict(x_test)
#yhat = model.predict(x_train)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

# Retorna uma matriz bidimensional que representa se houve falsos positivos ou negativos
# Quanto maior a quantidade do valor de um lado da matriz melhor o modelo
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 1]],

       [[1, 0],
        [0, 4]]], dtype=int64)

In [19]:
# Retorna a precisão do modelo
accuracy_score(ytrue, yhat)

1.0

In [14]:
colors = [(245,117,16), (117,245,16), (16,117,245)]  #Array que guarda cores
def prob_viz(res, actions, input_frame, colors):
    """ Função que mostra na tela a probabilidade de cada um dos sinais, em tempo real.
    Recebe os resultados com as probabilidades, os sinais, um frame e as cores a serem utilizadas.
    Retorna o mesmo frame com as palavras e suas respectivas probabilidades.
    """
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):   #Para cada sinal cria os textos e probabilidades
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num] + str(round(res[num],2)), (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [60]:
sequence = []      # Guarda os frames
sentence = []      # Guarda quais foram as previsões feitas pelo modelo para formar frases 
predictions = []   # Guarda as prediçoes feita para evitar problemas na transição de sinais
threshold = 0.6    # Mínimo para renderizar os resultados, evitando resultados de baixa confiança

#Acessa a webcam - valor (0) representa o hardware
cap = cv2.VideoCapture(0)

#Define o modelo utilizado
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #Seleciona o frame atual da webcam
        ret, frame = cap.read()

        #Processa previsoes
        image, results = mediapipe_detection(frame, holistic)
        
        #Desenha os pontos
        #draw_landmarks(image, results)
        
        #Previsoes
        #Pega os pontos que estao aparecendo na camera e inserem em um array
        #Cada sequencia é feita com os 30 ultimos frames que depois será passada para o modelo
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        #Verifica se há alguma mão no frame, para só então começar a previsão
        rh_onscreen = (sequence[-1][-63:]!=np.zeros(63)).all()
        lh_onscreen = (sequence[-1][-126:-63]!=np.zeros(63)).all()
        
        #if len(sequences)==30:
        if len(sequence) == 30 and (rh_onscreen or lh_onscreen):
            #expand_dims permite passar uma sequencia por vez
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            #Visualização
            #Checa se os ultimos 10 frames são do mesmo sinal, para evitar problemas na transição de sinais
            if np.unique(predictions[-5:])[0]==np.argmax(res): 
                predictions = []
                #Compara se a probabilidade de certo sinal é maior que o minimo para mostrar
                if res[np.argmax(res)] > threshold: #seria possivel fazer a media?
                    #tentativa de zerar a sequencia apos previsão correta - ajuda?
                    if len(sentence) > 0:
                        # Verifica se o sinal já não está incluido na frase
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            #Caso a frase fique muito grande, guarda apenas os ultimos valores
            if len(sentence) > 5:
                sentence = sentence[-5:]
        
            #Renderiza as probabilidades
            image = prob_viz(res, actions, image, colors)
        
        #Renderiza predições e a frase
        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        #Faz o display do frame
        if ret == True: 
            cv2.imshow('Webcam', image)

            #Encerra a webcam 
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

In [53]:
cap.release()
cv2.destroyAllWindows()

In [32]:
sequence[-1][-126:-63].shape

(63,)

In [28]:
testearray = np.zeros(1662)

In [47]:
sequence2 = []
for seq in sequence:
    sequence2.append(np.zeros(1662))

In [51]:
np.array(sequence2).shape

(30, 1662)

In [50]:
np.array(sequence).shape

(30, 1662)

In [57]:
res = model.predict(np.expand_dims(sequence, axis=0))[0]

array([1.8321335e-01, 1.5490651e-04, 8.1663173e-01], dtype=float32)